# **Übung 9a** Programmierung mit Python mit Anwendungen aus dem Maschinellen Lernen



## Aufgabe 1
Diese Übung baut auf der Basisübung auf und soll Möglichkeiten aufzeigen, wie Sie Ihr neuronales Netz noch effizienter trainieren können. Der Datensatz und ein Basismodell sind bereits vorgegeben.

Versuchen Sie das Modell weiter zu verbessern
(Effizienz + Genauigkeit).

Verändern Sie dabei nicht die Anzahl der Parameter, die Anzahl der Layer und behalten sie die LinearLayer bei.

Mögliche Bereiche zur Verbesserung:
- Allgemeine Verbesserung
 - Optimizer
 - Aktivierungsfunktionen
 - Dynamisches anpassen der Learning Rate
 - Batch Size anpassen
 - Gewichte initialisieren
 - Mehrmals mit denselben Parametern, aber verschiedenen Seeds trainieren.
- Overfitting mitigieren
 - Regularization
 - Batchnorm
 - Dropout



In [ ]:
!pip install imshowtools

In [ ]:
import tqdm
import numpy as np
import pandas as pd
import plotly.express as px
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from imshowtools import imshow as multi_imshow

BATCH_SIZE = 100
EPOCHS = 20
LEARNING_RATE = 0.001

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambda x: x/255.),
                                transforms.RandomAffine(degrees=5, translate=(0.1,0.1), scale=(0.9, 1.1)),
                                transforms.ColorJitter(brightness=0.2, saturation=0.99),  
                                transforms.Lambda(lambda x: torch.flatten(x))
                                ])

transform_test = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambda x: x/255.),
                                transforms.Lambda(lambda x: torch.flatten(x))
                                ])

target_transform = transforms.Compose([transforms.Lambda(lambda x:torch.LongTensor([x])),
                                transforms.Lambda(lambda x: (F.one_hot(x, num_classes = 10).float().flatten(),x))
                                ])

# Trainingsdaten herunterladen + Transformation anwenden
train_set = datasets.FashionMNIST('./data', download=True, train=True, transform=transform, target_transform=target_transform)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

# Testdaten herunterladen + Transformation anwenden
test_set = datasets.FashionMNIST('./data', download=True, train=False, transform=transform_test, target_transform=target_transform)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

data,(labels_one_hot,labels) = next(iter(test_loader))

print(data.size(),labels_one_hot.size())

multi_imshow(*data[:20].resize(20,28,28,1)*255)#, cmap='binary')


In [ ]:
import tqdm
import numpy as np
import pandas as pd
import plotly.express as px
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modell
class Model(nn.Module):
    def __init__(self, n_features, hidden1, hidden2, n_out):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(n_features, hidden1)
        self.linear2 = nn.Linear(hidden1, hidden2)
        self.linear3 = nn.Linear(hidden2, n_out)

    def forward(self, x):
        y1 = F.sigmoid(self.linear1(x))
        y2 = F.sigmoid(self.linear2(y1))
        y3 = self.linear3(y2)
        return y3

# Modell erstellen
model = Model(784,500,250,10) 
model.to(device)

# Optimierer und Loss Funktion auswählen
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

all_losses = []
progress_bar = tqdm.tqdm(range(EPOCHS), leave=False)

# Epochen durchlaufen
for epoch in progress_bar:
    losses_train = []
    losses_test = []
    total_train = 0
    # Modell trainieren
    for inputs_train, (target_train_oh, target_train) in train_loader:
        inputs_train = inputs_train.to(device)
        target_train_oh = target_train_oh.to(device)
        target_train = target_train.to(device)
        optimizer.zero_grad()
        y_train = model(inputs_train)
        loss = loss_fn(y_train, target_train_oh)

        loss.backward()
        
        optimizer.step()
        
        progress_bar.set_description(f'Loss: {loss.item():.3f}')
        
        losses_train.append(loss.item())
        total_train += target_train.size(0)

    # Modell testen
    with torch.no_grad():
        correct_test = 0
        total_test = 0
        for inputs_test, (target_test_oh, target_test) in test_loader:
          inputs_test = inputs_test.to(device)
          target_test_oh = target_test_oh.to(device)
          target_test = target_test.to(device)
          y_test = model(inputs_test)
          _, target_pred = torch.max(y_test.data, 1)
          total_test += target_test.size(0)
          correct_test += (target_pred == target_test.flatten()).sum().item()
          loss_test = loss_fn(y_test, target_test_oh)

          losses_test.append(loss_test.item())

    epoch_loss = sum(losses_train) / total_train
    epoch_loss_test = sum(losses_test) / total_test
    acc_test = 100.0 * correct_test / total_test
    lr = optimizer.param_groups[-1]['lr']

    all_losses.append([epoch, epoch_loss, epoch_loss_test,acc_test,lr])


# Ergebnisse anzeigen
df_loss = pd.DataFrame(np.array(all_losses), columns=['epoch', 'loss_train', 'loss_test','acc_test','lr'])

fig = px.line(df_loss, x='epoch', y=['loss_train','loss_test'], width=850, height=500, log_y=True)
fig.update_layout(
    xaxis_title="epoch",
    yaxis_title="loss",
)
fig.show()

fig = px.line(df_loss, x='epoch', y=['acc_test'], width=850, height=500)
fig.update_layout(
    xaxis_title="epoch",
    yaxis_title="acc",
)
fig.show()

fig = px.line(df_loss, x='epoch', y=['lr'], width=850, height=500, log_y=True)
fig.update_layout(
    xaxis_title="epoch",
    yaxis_title="learning_rate",
)
fig.show()